# Illustration of load profile vs actual load timeseries

In [ ]:
from energyclustering.data.public.data import get_data_reading_preprocessed
import altair as alt
import pandas as pd
alt.data_transformers.disable_max_rows()
from chart_util import big_chart

In [ ]:
data_df = get_data_reading_preprocessed()

In [ ]:
day_df = (
    data_df[['Consumption']]
    .assign(
        day = lambda x: x.index.get_level_values(1).date,
        time = lambda x: x.index.get_level_values(1).time,
    )
    .reset_index()
    .pivot_table(index = ['iID', 'day'], columns = 'time', values = 'Consumption')
)
day_df

In [ ]:
data_df 

In [ ]:
plot_df = data_df.assign(time = lambda x: pd.to_datetime(x.index.get_level_values(1).time, format='%H:%M:%S', exact = False), date =  lambda x: pd.to_datetime(x.index.get_level_values(1).date, format = '%Y-%m-%d', exact = False))
plot_df

In [ ]:
def big_chart(chart, fontsize = 20): 
    return chart.configure_axis(
            grid = False, 
        labelFontSize = 16,
        titleFontSize = 16, 
            # offset = 5, 
    ).configure_title(
        fontSize = 16
        ).configure_legend(
    titleFontSize=fontsize,
    labelFontSize=fontsize
    ).configure_view(
        strokeWidth=0
)

In [ ]:
meterIDs = data_df.index.get_level_values(0).unique()

In [ ]:
import numpy as np
meters_to_use = np.random.default_rng(2).choice(meterIDs, 30)
idx = pd.IndexSlice
subset_df = plot_df.loc[idx[meters_to_use, '2016-06-15'], :]
all_days1 = alt.Chart(subset_df.reset_index(), title = 'Load time series of different consumers on June 15th', height = 200).mark_line(size = 1, opacity = 0.5).encode(
    x = alt.X('time:T', title = None,  axis=alt.Axis(format='%H:%M')),
    y = alt.Y('Consumption', title = 'Load (in kWh)'),
    color = alt.Color('iID:N', scale = alt.Scale(scheme = 'rainbow'), legend = None),
).interactive(bind_x = False)

average_day = subset_df.groupby('time')['Consumption'].mean().to_frame('Consumption')
avg_chart1 = alt.Chart(average_day.reset_index(), title = 'Averaged daylong load time series of 15th June').mark_line(size = 3.5).encode(
       x = alt.X('time:T', title = None,  axis=alt.Axis(format='%H:%M')),
    y = alt.Y('Consumption', title = 'Load (in kWh)'),
)
(all_days1 | avg_chart1).resolve_scale(y = 'shared', x = 'shared')

In [ ]:
IDX = 4
idx = pd.IndexSlice
subset_df = plot_df.loc[idx[meterIDs[IDX], pd.to_datetime('2016-06-01'):pd.to_datetime('2016-06-30')], :]
all_days = alt.Chart(subset_df.reset_index(), title = 'Load time series in June of a single consumer', height = 200).mark_line(size = 1, opacity = 0.5).encode(
    x = alt.X('time:T', title = None,  axis=alt.Axis(format='%H:%M')), 
    y = alt.Y('Consumption', title = 'Load (in kWh)'),
    color = alt.Color('date:N', scale = alt.Scale(scheme = 'rainbow'), legend = None),
).interactive(bind_x = False)

average_day = subset_df.groupby('time')['Consumption'].mean().to_frame('Consumption')
avg_chart = alt.Chart(average_day.reset_index(), title = 'Average daylong load time series of a single consumer in June').mark_line(size = 3.5).encode(
       x = alt.X('time:T', title = None,  axis=alt.Axis(format='%H:%M')),
   y = alt.Y('Consumption', title = 'Load (in kWh)'),
)
final_chart = ((all_days + avg_chart).resolve_scale(y = 'shared') | (all_days1 + avg_chart1).resolve_scale(y = 'shared')).resolve_scale(color = 'independent', y = 'shared', x = 'shared')
big_chart(final_chart)